In [1]:
!pip install selenium
!pip install chromedriver-autoinstaller
!pip install pandas
!pip install requests
!pip install bs4

In [2]:
import pandas as pd
import time
import requests
import chromedriver_autoinstaller

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [3]:
service = Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

KeyboardInterrupt: 

In [ ]:
driver.get("https://anmlegis.datalegis.net/action/ActionDatalegis.php?acao=apresentacao&cod_modulo=351&cod_menu=8014")
time.sleep(5)

In [ ]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/a[4]').click()
#clica em "Títulos Outorgados"

In [ ]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div/div[2]/div[4]/ul/li[5]/a').click()
time.sleep(5)
#clica em "Permissão de Lavra Garimpeira - PLG"

In [ ]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul/li[1]/a').click()
time.sleep(5)
#clica em "2024"

In [ ]:
last_height2024 = driver.execute_script("return document.body.scrollHeight")

In [ ]:
for vez2024 in range (3):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    new_height2024 = driver.execute_script("return document.body.scrollHeight")

    if new_height2024 == last_height2024 :
        break   
    last_height2024 = new_height2024

In [ ]:
soup2024 = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
tabela2024 = soup2024.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [ ]:
titulos2024 = tabela2024.find_all('strong', {})
conteudos2024 = tabela2024.find_all('p')[0:150:2]
datasPublicacao2024 = tabela2024.find_all('p', {'class': 'data-hora'})

In [ ]:
lista_titulos2024 = []
lista_conteudos2024 = []
lista_datasPublicacao2024 = []

for titulo2024 in titulos2024:
  lista_titulos2024.append(titulo2024.get_text().strip())

for conteudo2024 in conteudos2024:
  lista_conteudos2024.append(conteudo2024.get_text().strip())

for dataPublicacao2024 in datasPublicacao2024:
  lista_datasPublicacao2024.append(dataPublicacao2024.get_text().strip())

df_plg2024 = pd.DataFrame([lista_titulos2024, lista_conteudos2024, lista_datasPublicacao2024]).T

In [ ]:
driver.back()

In [ ]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul/li[2]/a').click()
time.sleep(5)
#clica em "2023"

In [ ]:
last_height2023 = driver.execute_script("return document.body.scrollHeight")

In [ ]:
for vez2023 in range (7):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    new_height2023 = driver.execute_script("return document.body.scrollHeight")

    if new_height2023 == last_height2023 :
        break   
    last_height2023 = new_height2023

In [ ]:
soup2023 = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
tabela2023 = soup2023.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [ ]:
titulos2023 = tabela2023.find_all('strong', {})
conteudos2023 = tabela2023.find_all('p')[0:250:2]
datasPublicacao2023 = tabela2023.find_all('p', {'class': 'data-hora'})

In [ ]:
lista_titulos2023 = []
lista_conteudos2023 = []
lista_datasPublicacao2023 = []

for titulo2023 in titulos2023:
  lista_titulos2023.append(titulo2023.get_text().strip())

for conteudo2023 in conteudos2023:
  lista_conteudos2023.append(conteudo2023.get_text().strip())

for dataPublicacao2023 in datasPublicacao2023:
  lista_datasPublicacao2023.append(dataPublicacao2023.get_text().strip())

df_plg2023 = pd.DataFrame([lista_titulos2023, lista_conteudos2023, lista_datasPublicacao2023]).T

In [ ]:
driver.back()

In [ ]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul/li[3]/a').click()
time.sleep(5)
#clica em "2022"

In [ ]:
last_height2022 = driver.execute_script("return document.body.scrollHeight")

In [ ]:
for vez2022 in range (14):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    new_height2022 = driver.execute_script("return document.body.scrollHeight")

    if new_height2022 == last_height2022 :
        break   
    last_height2022 = new_height2022

In [ ]:
soup2022 = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
tabela2022 = soup2022.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [ ]:
titulos2022 = tabela2022.find_all('strong', {})
conteudos2022 = tabela2022.find_all('p')[0:600:2]
datasPublicacao2022 = tabela2022.find_all('p', {'class': 'data-hora'})

In [ ]:
lista_titulos2022 = []
lista_conteudos2022 = []
lista_datasPublicacao2022 = []

for titulo2022 in titulos2022:
  lista_titulos2022.append(titulo2022.get_text().strip())

for conteudo2022 in conteudos2022:
  lista_conteudos2022.append(conteudo2022.get_text().strip())

for dataPublicacao2022 in datasPublicacao2022:
  lista_datasPublicacao2022.append(dataPublicacao2022.get_text().strip())

df_plg2022 = pd.DataFrame([lista_titulos2022, lista_conteudos2022, lista_datasPublicacao2022]).T

In [ ]:
driver.back()

In [ ]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul[1]/li[5]/a').click()
time.sleep(5)
#clica em "2021"

In [ ]:
last_height2021 = driver.execute_script("return document.body.scrollHeight")

In [ ]:
for vez2021 in range (6):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    new_height2021 = driver.execute_script("return document.body.scrollHeight")

    if new_height2021 == last_height2021 :
        break   
    last_height2021 = new_height2021

In [ ]:
soup2021 = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
tabela2021 = soup2021.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [ ]:
titulos2021 = tabela2021.find_all('strong', {})
conteudos2021 = tabela2021.find_all('p')[0:250:2]
datasPublicacao2021 = tabela2021.find_all('p', {'class': 'data-hora'})

In [ ]:
lista_titulos2021 = []
lista_conteudos2021 = []
lista_datasPublicacao2021 = []

for titulo2021 in titulos2021:
  lista_titulos2021.append(titulo2021.get_text().strip())

for conteudo2021 in conteudos2021:
  lista_conteudos2021.append(conteudo2021.get_text().strip())

for dataPublicacao2021 in datasPublicacao2021:
  lista_datasPublicacao2021.append(dataPublicacao2021.get_text().strip())

df_plg2021 = pd.DataFrame([lista_titulos2021, lista_conteudos2021, lista_datasPublicacao2021]).T

In [ ]:
driver.back()

In [ ]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul/li[5]/a').click()
time.sleep(5)
#clica em "2020"

In [ ]:
last_height2020 = driver.execute_script("return document.body.scrollHeight")

In [ ]:
for vez2020 in range (11):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    new_height2020 = driver.execute_script("return document.body.scrollHeight")

    if new_height2020 == last_height2020 :
        break   
    last_height2020 = new_height2020

In [ ]:
soup2020 = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
tabela2020 = soup2020.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [ ]:
titulos2020 = tabela2020.find_all('strong', {})
conteudos2020 = tabela2020.find_all('p')[0:450:2]
datasPublicacao2020 = tabela2020.find_all('p', {'class': 'data-hora'})

In [ ]:
lista_titulos2020 = []
lista_conteudos2020 = []
lista_datasPublicacao2020 = []

for titulo2020 in titulos2020:
  lista_titulos2020.append(titulo2020.get_text().strip())

for conteudo2020 in conteudos2020:
  lista_conteudos2020.append(conteudo2020.get_text().strip())

for dataPublicacao2020 in datasPublicacao2020:
  lista_datasPublicacao2020.append(dataPublicacao2020.get_text().strip())

df_plg2020 = pd.DataFrame([lista_titulos2020, lista_conteudos2020, lista_datasPublicacao2020]).T

In [ ]:
driver.back()

In [ ]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul/li[6]/a').click()
time.sleep(5)
#clica em "2019"

In [ ]:
last_height2019 = driver.execute_script("return document.body.scrollHeight")

In [ ]:
for vez2019 in range (5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    new_height2019 = driver.execute_script("return document.body.scrollHeight")

    if new_height2019 == last_height2019 :
        break   
    last_height2019 = new_height2019

In [ ]:
soup2019 = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
tabela2019 = soup2019.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [ ]:
titulos2019 = tabela2019.find_all('strong', {})
conteudos2019 = tabela2019.find_all('p')[0:210:2]
datasPublicacao2019 = tabela2019.find_all('p', {'class': 'data-hora'})

In [ ]:
lista_titulos2019 = []
lista_conteudos2019 = []
lista_datasPublicacao2019 = []

for titulo2019 in titulos2019:
  lista_titulos2019.append(titulo2019.get_text().strip())

for conteudo2019 in conteudos2019:
  lista_conteudos2019.append(conteudo2019.get_text().strip())

for dataPublicacao2019 in datasPublicacao2019:
  lista_datasPublicacao2019.append(dataPublicacao2019.get_text().strip())

df_plg2019 = pd.DataFrame([lista_titulos2019, lista_conteudos2019, lista_datasPublicacao2019]).T

In [ ]:
driver.back()

In [ ]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul/li[7]/a').click()
time.sleep(5)
#clica em "2018"

In [ ]:
soup2018 = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
tabela2018 = soup2018.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [ ]:
titulos2018 = tabela2018.find_all('strong', {})
conteudos2018 = tabela2018.find_all('p')[0:8:2]
datasPublicacao2018 = tabela2018.find_all('p', {'class': 'data-hora'})

In [ ]:
lista_titulos2018 = []
lista_conteudos2018 = []
lista_datasPublicacao2018 = []

for titulo2018 in titulos2018:
  lista_titulos2018.append(titulo2018.get_text().strip())

for conteudo2018 in conteudos2018:
  lista_conteudos2018.append(conteudo2018.get_text().strip())

for dataPublicacao2018 in datasPublicacao2018:
  lista_datasPublicacao2018.append(dataPublicacao2018.get_text().strip())

df_plg2018 = pd.DataFrame([lista_titulos2018, lista_conteudos2018, lista_datasPublicacao2018]).T

In [ ]:
plg = pd.concat([df_plg2018, df_plg2019, df_plg2020, df_plg2021, df_plg2022, 
                df_plg2023, df_plg2024])

In [ ]:
plg

,0,1,2
0,LICENCIAMENTO Nº 35/2018,"Outorga o Licenciamento, em nome de JOSE DAMIÃ...",11/05/2018 | 07:25:20
1,LICENCIAMENTO Nº 001/2018,"Outorga o Licenciamento, em nome de AREIA DA L...",01/02/2018 | 12:28:49
0,PERMISSÃO DE LAVRA GARIMPEIRA Nº 187/2019 DE 5...,"Outorga pelo prazo de 5 ano(s), COOPERATIVA DE...",09/12/2019 | 10:39:27
1,PERMISSÃO DE LAVRA GARIMPEIRA Nº 186/2019 DE 2...,"Outorga pelo prazo de 5 ano(s), COOPERATIVA DE...",04/12/2019 | 10:36:45
2,PERMISSÃO DE LAVRA GARIMPEIRA Nº 185/2019 DE 2...,"Outorga pelo prazo de 5 ano(s), RIBEIRO OMAR D...",04/12/2019 | 10:34:15
...,...,...,...
40,"PERMISSÃO DE LAVRA GARIMPEIRA N° 6/2024, de 23...","Outorga a JOÃO EVANGELISTA ALVES, pelo prazo d...",25/01/2024 | 11:31:45
41,"PERMISSÃO DE LAVRA GARIMPEIRA N° 5/2024, de 23...","Outorga a JOÃO EVANGELISTA ALVES, pelo prazo d...",25/01/2024 | 11:29:38
42,PERMISSÃO DE LAVRA GARIMPEIRA Nº 1/2024,"Outorga até POR 05 (CINCO) ANOS, a Robson Lana...",19/01/2024 | 08:59:28
43,PERMISSÃO DE LAVRA GARIMPEIRA Nº 4/2024,"Outorga POR 05 (CINCO) ANOS, a Sandro Andrade ...",17/01/2024 | 10:07:59
